## Modules

In [3]:
# download relevant modules
# ! pip install time
# ! pip install numpy
# ! pip install pandas
# ! pip install yfinance
# ! pip install matplotlib
# ! pip install PyPortfolioOpt

In [4]:
# include libraries
import io
import time
import contextlib
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

In [5]:
# ignore system warnings
import warnings
warnings.filterwarnings("ignore")

## Portfolio

In [162]:
def snp500():

  url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
  companies = pd.read_html(url)[0][0:25]
  return companies['Symbol'], companies['Security']

In [188]:
def Information():

    # input stock symbols (tickers)
    #companies = ['META', 'AMZN', 'AAPL', 'MSFT', 'GOOG', 'TSLA']
    companies, names = snp500()

    # assign weight to each stock (total weight should be one) 
    weights = np.full(len(companies), 1/len(companies)) # (companies, weight value)

    # portfolio start date
    stockStartDate = '2015-01-01' # (Y, M, D)

    # portfolio end date
    today = datetime.today().strftime('%Y-%m-%d')
    print('Portfolio based on data from ' + stockStartDate + " to " + today)
    print("Total companies listed on portfolio : ", len(companies))

    return companies, weights, stockStartDate, today, names

In [189]:
def Dataset(companies, stockStartDate, today):

    # download companies' stock prices
    data = pd.DataFrame()

    # stop any messages from yahoo finance api
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
      
      for stock in companies:
        data[stock] = yf.download(stock, start=stockStartDate, end=today)['Adj Close']

    return data.fillna(0)

In [190]:
# visualize prices
#data.plot()

In [191]:
def Analysis(data, weights):

    # record start time
    start = time.time()

    # daily returns
    returns = data.pct_change()

    # display annual covariance matrix
    trading_days = 252 # count of trading days this year
    annual_covariance = returns.cov() * trading_days # diagonals (variance), others (covariance)

    # calculate portfolio variance
    portfolio_variance = np.dot(weights.T, np.dot(annual_covariance, weights)) 

    # calculate portfolio standard-deviation (volatility)
    portfolio_volatility = np.sqrt(portfolio_variance)

    # calculate portfolio returns
    portfolio_returns = np.sum(returns.mean() * weights) * trading_days

    # record start time
    end = time.time()

    # total time of analysis
    total_time = end - start

    # display
    print("Expected Annual Portfolio Performance \n")
    print("1.  Returns : ", str(round(portfolio_returns, 3) * 100) + '%')
    print("2.  Risk(Volatility) : ", str(round(portfolio_volatility, 3) * 100) + '%')
    print("3.  Variance : ", str(round(portfolio_variance, 3) * 100) + '%')
    print("4.  Time Taken :", total_time, " seconds")

In [192]:
companies, weights, stockStartDate, today, names = Information()
data = Dataset(companies, stockStartDate, today)
Analysis(data, weights)

Portfolio based on data from 2015-01-01 to 2023-03-11
Total companies listed on portfolio :  25
Expected Annual Portfolio Performance 

1.  Returns :  14.899999999999999%
2.  Risk(Volatility) :  19.0%
3.  Variance :  3.5999999999999996%
4.  Time Taken : 0.006031990051269531  seconds


## Optimization

In [193]:
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [194]:
def Optimization(data):

    r = expected_returns.mean_historical_return(data)
    v = risk_models.sample_cov(data)

    EF = EfficientFrontier(r, v) # provide highest expected return with minimum level of risk
    weights = EF.max_sharpe() # provides the best trade-off between risk and return
    weights = EF.clean_weights() # remove any small weights that may have resulted from rounding errors

    print("Optimized Portfolio : ", weights)
    print("\n----------Optimized Portfolio Performance----------")
    EF.portfolio_performance(verbose = True)

    return weights

In [195]:
def Portfolio(data, weights):

    investment = 10000
    latest_prices = get_latest_prices(data) # last day price
    DA = DiscreteAllocation(weights, latest_prices, total_portfolio_value=investment) # allocating based on percentages

    allocated, remaining = DA.lp_portfolio() # allocations
    print("\n----------Your Portfolio----------")
    print("Amount Invested : ", investment)
    print("Stocks Allocated To Portfolio : ", allocated)
    print("Funds Remaining : $", remaining)

In [196]:
weights = Optimization(data)
Portfolio(data, weights)

Optimized Portfolio :  OrderedDict([('MMM', 0.0), ('AOS', 0.0), ('ABT', 0.0), ('ABBV', 0.23073), ('ACN', 0.05896), ('ATVI', 0.22768), ('ADM', 0.0), ('ADBE', 0.1145), ('ADP', 0.05846), ('AAP', 0.0), ('AES', 0.0), ('AFL', 0.02857), ('A', 0.13167), ('APD', 0.0), ('AKAM', 0.0), ('ALK', 0.0), ('ALB', 0.06869), ('ARE', 0.0), ('ALGN', 0.05257), ('ALLE', 0.0), ('LNT', 0.01728), ('ALL', 0.0), ('GOOGL', 0.0), ('GOOG', 0.01087), ('MO', 0.0)])

----------Optimized Portfolio Performance----------
Expected annual return: 17.4%
Annual volatility: 20.9%
Sharpe Ratio: 0.74

----------Your Portfolio----------
Amount Invested :  10000
Stocks Allocated To Portfolio :  {'ABBV': 15, 'ACN': 2, 'ATVI': 29, 'ADBE': 3, 'ADP': 3, 'AFL': 4, 'A': 10, 'ALB': 3, 'ALGN': 2, 'LNT': 3, 'GOOG': 1}
Funds Remaining : $ 202.86987569155826


In [197]:
portfolio = pd.DataFrame()
portfolio['Name'] = names
portfolio['Symbol'], portfolio['weightage'] = weights.keys(), weights.values()

portfolio

,Name,Symbol,weightage
0,3M,MMM,0.00000
1,A. O. Smith,AOS,0.00000
2,Abbott,ABT,0.00000
3,AbbVie,ABBV,0.23073
4,Accenture,ACN,0.05896
5,Activision Blizzard,ATVI,0.22768
6,ADM,ADM,0.00000
7,Adobe Inc.,ADBE,0.11450
8,ADP,ADP,0.05846
9,Advance Auto Parts,AAP,0.00000
